In [3]:
import pandas as pd
import numpy as np

In [19]:
# read in the data
books = pd.read_csv("/Users/kapilojha/Desktop/ml/projects/book-recommender/data/books.csv",
                encoding="latin-1",
                on_bad_lines="skip",
                sep=";"
        )

/var/folders/mw/0qwwq_0146j20zhtp_s1sb4c0000gn/T/ipykernel_15867/3863721045.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/Users/kapilojha/Desktop/ml/projects/book-recommender/data/books.csv",


In [7]:
!ls

books_data     research.ipynb


In [20]:
# view the data
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [24]:
print("Large URl: ", books["Image-URL-L"].iloc[0])
print("Medium URl: ", books["Image-URL-M"].iloc[0])
print("Small URl: ", books["Image-URL-S"].iloc[0])

Large URl:  http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
Medium URl:  http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg
Small URl:  http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg


Each book has 3 diffrent Image url :
- depending on our requirements we can choose any of the three
- small, medium and large.

In [25]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [27]:
books.shape

(271360, 8)

We have total of 2,71,360 rows and 8 features.

Since we do not need the small and medium url's we should remove them from
our dataset.

In [28]:
books.drop(["Image-URL-S", "Image-URL-M"],
           axis=1,
           inplace=True)
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [29]:
# rename the column names for easier processing
books.rename(columns={
    "ISBN" : "isbn",
    "Book-Title" : "title",
    "Book-Author" : "author",
    "Year-Of-Publication" : "year",
    "Publisher" : "publisher",
    "Image-URL-L" : "image-url"
}, inplace=True)
books.head()

,isbn,title,author,year,publisher,image-url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
